## Clustering the Neighborhoods of Toronto 

#### Introduction

The following exercise is based on segmenting and clustering the neighborhoods in the city of Toronto, Canada. The data about canada is obtained by web scrapping the Wikipedia page available in the discription for the excercise. The coordinate data of latitude and longitude for the location is used to plot map and pointers with the help of Folium library. Foursquare API is used to retrieve the data of popular venues. 

### Import the Libraries 

In [3]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium


!conda install -c conda-forge lxml --yes
import lxml

from sklearn.cluster import KMeans

print('Libraries Imported')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

### Web scrab the wikipedia page to obtain the data

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
df_list = pd.read_html(results.text)
df = df_list[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Drop the rows which are not assigned items in the column "Brorough"

In [5]:
df.drop(df[df.Borough == "Not assigned"].index,inplace=True)
neighborhood_df = df.reset_index(drop=True)
neighborhood_df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Using the pandas library read the geospatial coordinates from CSV file

In [6]:
geo_df = pd.read_csv("Geospatial_Coordinates.csv")
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Merge two tables into a single data frame

In [7]:
df_3 = pd.merge(neighborhood_df,geo_df)
df_3

df_3

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Reset the index by updating the table with regions of neighborhood of toronto in column "Borough".

In [8]:
toronto_data = df_3[df_3["Borough"].str.contains("Toronto")].reset_index(drop=True)
toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


### Use geopy library to get the latitude and longitude values of New York City.

The user_agent assumed in the geolocator is "TO_explorer"

In [12]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent='TO_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Latitude : ',latitude)
print('Longitude : ',longitude)

Latitude :  43.6534817
Longitude :  -79.3839347


### Create a map of Toronto with neighborhoods superimposed on top using Folium Library.

In [13]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=12)

for lat,lng,label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    color = 'blue',
    fill = True,
    fill_color='#3189cc',
    parse_html=False).add_to(map_toronto)
    
map_toronto


### Lets draw our focus towards the Ceteral Toronto to explore the important venues in this location

In [14]:
centeral_toronto_data = toronto_data[toronto_data['Borough'] == 'Central Toronto'].reset_index(drop = True)
centeral_toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
5,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
6,M4S,Central Toronto,Davisville,43.704324,-79.388790
7,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [15]:
address = 'Central Toronto, ON, Canada'

geolocator = Nominatim(user_agent='TO_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Latitude : ',latitude)
print('Longitude : ',longitude)

Latitude :  43.6534817
Longitude :  -79.3839347


In [16]:
map_centeral = folium.Map(location=[latitude,longitude],zoom_start=11)

for lat,lng,label in zip(centeral_toronto_data['Latitude'], centeral_toronto_data['Longitude'], centeral_toronto_data['Neighborhood']):
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    color = 'blue',
    fill = True,
    fill_color='#3189cc',
    parse_html=False).add_to(map_centeral)
    
map_centeral

### Let's explore the first neighborhood in our dataframe.

In [17]:
centeral_toronto_data.loc[0,'Neighborhood']

'Lawrence Park'

In [20]:
neighborhood_latitude = centeral_toronto_data.loc[0, 'Latitude']
neighborhood_longitude = centeral_toronto_data.loc[0, 'Longitude'] 

neighborhood_name = centeral_toronto_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


### Define Foursquare Credentials and Version

In [21]:
CLIENT_ID = 'DUM4H0JRAWKCTZAO1Y4XH5L2333AD1O1UY50AHU1UBRAYIXK'
CLIENT_SECRET = 'SB5S11VC2QSE5FRQ14AXNS5W1SWITI2KP30Z2WL3KXWJGUYE'
VERSION = '20200608'
LIMIT = 100
radius = 500

print('Client_Id : ',CLIENT_ID)
print('Client_Secret : ',CLIENT_SECRET)
print('Version : ',VERSION)

Client_Id :  DUM4H0JRAWKCTZAO1Y4XH5L2333AD1O1UY50AHU1UBRAYIXK
Client_Secret :  SB5S11VC2QSE5FRQ14AXNS5W1SWITI2KP30Z2WL3KXWJGUYE
Version :  20200608


### Explore Neighborhoods in Centeral Toronto

Let's create a function to repeat the same process to all the neighborhoods in Central Toronto

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
centeral_toronto_venues = getNearbyVenues(names=centeral_toronto_data['Neighborhood'],latitudes=centeral_toronto_data['Latitude'],longitudes=centeral_toronto_data['Longitude'])

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


In [24]:
print(centeral_toronto_venues.shape)
centeral_toronto_venues.head()

(110, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden


In [25]:
centeral_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,31,31,31,31,31,31
Davisville North,9,9,9,9,9,9
"Forest Hill North & West, Forest Hill Road Park",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",2,2,2,2,2,2
"North Toronto West, Lawrence Park",19,19,19,19,19,19
Roselawn,3,3,3,3,3,3
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",17,17,17,17,17,17
"The Annex, North Midtown, Yorkville",22,22,22,22,22,22


In [26]:
print('There are {} unique categories in centeral toronto'.format(len(centeral_toronto_venues['Venue Category'].unique())))

There are 61 unique categories in centeral toronto


In [27]:
CT_onehot = pd.get_dummies(centeral_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

CT_onehot['Neighborhood'] = centeral_toronto_venues['Neighborhood'] 

fixed_columns = [CT_onehot.columns[-1]] + list(CT_onehot.columns[:-1])
CT_onehot = CT_onehot[fixed_columns]

CT_onehot.head(5)

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
CT_onehot.shape

(110, 62)

In [29]:
CT_grouped = CT_onehot.groupby('Neighborhood').mean().reset_index()
CT_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.064516,0.000000,0.000000,0.064516,0.000000,0.000000,0.096774,0.032258,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.032258,0.032258,0.032258,0.000000,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.064516,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.032258,0.064516,0.000000,0.000000,0.032258,0.000000,0.096774,0.032258,0.000000,0.000000,0.000000,0.000000,0.064516,0.000000,0.032258,0.032258,0.00,0.000000,0.000000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Forest Hill North & West, Forest Hill Road Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,"North Toronto West, Lawrence Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.052632,0.157895,0.105263,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0

In [30]:
CT_grouped.shape

(9, 62)

In [31]:
num_top_venues = 5

for hood in CT_grouped['Neighborhood']:
    print('----'+hood+'----')
    temp = CT_grouped[CT_grouped['Neighborhood']== hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0        Dessert Shop  0.10
1      Sandwich Place  0.10
2  Italian Restaurant  0.06
3                 Gym  0.06
4                Café  0.06


----Davisville North----
                  venue  freq
0                 Hotel  0.11
1      Department Store  0.11
2  Gym / Fitness Center  0.11
3                   Gym  0.11
4                  Park  0.11


----Forest Hill North & West, Forest Hill Road Park----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2   Mexican Restaurant  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0             Bus Line  0.33
1          Swim School  0.33
2                 Park  0.33
3  American Restaurant  0.00
4           Restaurant  0.00


----Moore Park, Summerhill East----
                 venue  freq
0                  Gym   0.5
1                 Park   0.5
2  American Restaurant   0.0
3           Restaurant

In [32]:
def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Explore the 10 venues of Lawrence Park

In [33]:
num_top_venues = 10

indicators = ['st','nd','rd']

columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most common venues'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most common venues'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns) 
neighborhoods_venues_sorted['Neighborhood'] = CT_grouped['Neighborhood']

for ind in np.arange(CT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(CT_grouped.iloc[1,:],num_top_venues)
    
neighborhoods_venues_sorted.head(50)

,Neighborhood,1st Most common venues,2nd Most common venues,3rd Most common venues,4th Most common venues,5th Most common venues,6th Most common venues,7th Most common venues,8th Most common venues,9th Most common venues,10th Most common venues
0,Davisville,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
1,Davisville North,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
2,"Forest Hill North & West, Forest Hill Road Park",Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
3,Lawrence Park,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
4,"Moore Park, Summerhill East",Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
5,"North Toronto West, Lawrence Park",Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
6,Roselawn,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
7,"Summerhill West, Rathnelly, South Hill, Forest...",Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
8,"The Annex, North Midtown, Yorkville",Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line


### Perform Clustering

Choosing the number of clusters to be 4 and Kmeans method to perform clustering

In [34]:
kclusters = 4

CT_grouped_clustering = CT_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(CT_grouped_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 2, 1, 0, 3, 0, 0], dtype=int32)

In [44]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

CT_merged = centeral_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
CT_merged = CT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
CT_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most common venues,2nd Most common venues,3rd Most common venues,4th Most common venues,5th Most common venues,6th Most common venues,7th Most common venues,8th Most common venues,9th Most common venues,10th Most common venues
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line


In [45]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [46]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CT_merged['Latitude'], CT_merged['Longitude'], CT_merged['Neighborhood'], CT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [47]:
CT_merged.loc[CT_merged['Cluster Labels'] == 0,CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most common venues,2nd Most common venues,3rd Most common venues,4th Most common venues,5th Most common venues,6th Most common venues,7th Most common venues,8th Most common venues,9th Most common venues,10th Most common venues
2,Central Toronto,0,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
3,Central Toronto,0,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
4,Central Toronto,0,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
5,Central Toronto,0,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
6,Central Toronto,0,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
8,Central Toronto,0,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line


In [48]:
CT_merged.loc[CT_merged['Cluster Labels'] == 1,CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most common venues,2nd Most common venues,3rd Most common venues,4th Most common venues,5th Most common venues,6th Most common venues,7th Most common venues,8th Most common venues,9th Most common venues,10th Most common venues
7,Central Toronto,1,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line


In [49]:
CT_merged.loc[CT_merged['Cluster Labels'] == 2,CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most common venues,2nd Most common venues,3rd Most common venues,4th Most common venues,5th Most common venues,6th Most common venues,7th Most common venues,8th Most common venues,9th Most common venues,10th Most common venues
0,Central Toronto,2,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line


In [50]:
CT_merged.loc[CT_merged['Cluster Labels'] == 3,CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most common venues,2nd Most common venues,3rd Most common venues,4th Most common venues,5th Most common venues,6th Most common venues,7th Most common venues,8th Most common venues,9th Most common venues,10th Most common venues
1,Central Toronto,3,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
